In [12]:
from helium import *
import time
import numpy as np
import pandas as pd
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import threading

In [13]:
def handle_row_one_bigger(row1,row2):
    counter = len(row1) + len(row2)
    courses_of_row1 = [item.web_element.text for item in row1]
    courses_of_row2 = [item.web_element.text for item in row2]
    
    temp = pd.DataFrame({'course_code':[],'course_name':[],'section_number':[],'course_type':[],
                        'course_hours':[],'course_gender':[]})
    
    for i in range(len(courses_of_row1)-1):
        temp_lecture_df = Extracter(courses_of_row1[i])
        temp_tutorial_df = Extracter(courses_of_row2[i])
        temp = pd.concat([temp,temp_lecture_df])
        temp = pd.concat([temp,temp_tutorial_df])
    temp= temp.append(Extracter(courses_of_row1[-1]))
    return temp
        

In [14]:
def handle_row_two_bigger(row1,row2):
    counter = len(row1) + len(row2)
    courses_of_row1 = [item.web_element.text for item in row1]
    courses_of_row2 = [item.web_element.text for item in row2]
    
    temp = pd.DataFrame({'course_code':[],'course_name':[],'section_number':[],'course_type':[],
                        'course_hours':[],'course_gender':[]})
    
    for i in range(len(courses_of_row2)-1):
        temp_lecture_df = Extracter(courses_of_row1[i])
        temp_tutorial_df = Extracter(courses_of_row2[i])
        temp = pd.concat([temp,temp_tutorial_df])
        temp = pd.concat([temp,temp_lecture_df])
        
    temp= temp.append(Extracter(courses_of_row2[-1]))
    return temp
        

In [15]:
def Extracter(row):
   
    #Lecture extracter
    row=row.split(" ")
    if row[-5] == "محاضرة" or row[-5] == 'تدريب' or row[-5] == 'مشروع':
        course_code = row[0] +" "+ row[1]
        course_name = " ".join(row[2:-6])
        section_number = row[-6]
        course_type = row[-5]
        course_hours = row[-4]
        course_gender = row[-3]
        temp_df = pd.DataFrame({'course_code':[course_code],'course_name':[course_name],'section_number':[section_number],
                                'course_type':[course_type],'course_hours':[course_hours],'course_gender':[course_gender],
                               },index=[0])
        #print(temp_df)
        return temp_df
    
    elif row[-4] == 'تمارين' or row[-4] == "عملي":
        course_code = row[0] +" "+ row[1]
        course_name = " ".join(row[2:-5])
        section_number = row[-5]
        course_type = row[-4]
        course_hours = row[-3]
        course_gender = row[-2]
        temp_df  = pd.DataFrame({'course_code':[course_code],'course_name':[course_name],'section_number':[section_number],
                                'course_type':[course_type],'course_hours':[course_hours],'course_gender':[course_gender]},index=[0])
        return temp_df
    
    else:
        temp_df = pd.DataFrame({'course_code':['redundant'],'course_name':['redundant'],'section_number':['redundant'],'course_type':['redundant'],
                          'course_hours':['redundant'],'course_gender':['redundant']})
        return temp_df

In [16]:
def ExtractCourseDay(text):
    digits_only = re.findall('\d', text)
    return digits_only

# Scrape DataFrame for saving and cleaning later

In [17]:
courses_df = pd.DataFrame({'course_code':[],'course_name':[],'section_number':[],'course_type':[],
                          'course_hours':[],'course_gender':[]})

In [18]:
finals_df = pd.DataFrame({'course_day':[],'course_time':[],'final_exam_day':[],
                          'final_exam_date':[],'final_exam_time':[],'lecturer':[]})

In [19]:
start_chrome()
go_to("https://edugate.ksu.edu.sa/ksu/ui/guest/timetable/index/mainScheduleTreeCoursesIndex.faces")
#click("أكاديمي")
#time.sleep(1)
#click("المقررات المطروحة")
#time.sleep(1)
#click("أكاديمي")
#time.sleep(1)
click(Link("المقررات المطروحة",above="المقررات المطروحة - بحث متقدم"))
time.sleep(1)
click("علوم الحاسب والمعلومات")
time.sleep(1)
click("هندسة البرمجيات")
time.sleep(1)
click("SWEN-هندسة البرمجيات")
time.sleep(1)
click("إبحث")
for i in range(11):
    click(S(".pui-paginator-next"))
counter = 1
while counter < 11:
    
    # Get all of the courses 
    time.sleep(1)
    row1 = find_all(S('.ROW1'))
    row2 = find_all(S('.ROW2'))
    time.sleep(1)
    if len(row1) > len(row2):
        courses_df = courses_df.append(handle_row_one_bigger(row1,row2))
    
    else:
        courses_df = courses_df.append(handle_row_two_bigger(row1,row2))
    
    # Get Final Exam, Course Day and Course Time
    #time.sleep(1)
    #links = find_all((Link("التفاصيل")))
    #time.sleep(1)
    #pattern = r"\d+"
    #first = str(links[0])
    # find all occurrences of the pattern in the text
    #firstInList = int(re.findall(pattern, first)[0])
    #second = str(links[-1])
    #secondInList = int(re.findall(pattern,second)[0])
    # print the matches
    """"
    row = None
    if firstInList>secondInList:
        for i in reversed(links):
            time.sleep(1)
            click(i)
            time.sleep(1)
            courseDay = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='اليوم'))
            courseTime = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='الوقت'))
            courseDay = [item.web_element.text for item in courseDay]
            courseTime = [item.web_element.text for item in courseTime]
            #instructorDiv
            #courseDay = ExtractCourseDay(str(row[0]))
            #courseTime = ExtractCourseDayAndDuration(row)
            examDay = [item.web_element.text for item in find_all(S("#examDayDiv"))]
            examTime = [item.web_element.text for item in find_all(S("#examTimeDiv"))]
            examDate = [item.web_element.text for item in find_all(S("#examDateDiv"))]
            lecturer = [item.web_element.text for item in find_all(S("#instructorDiv"))]
            temp_df = pd.DataFrame({'course_day':[courseDay],'course_time':[courseTime],
            'final_exam_day':[examDay[0]], 'final_exam_date':[examDate[0]],'final_exam_time':[examTime[0]],
                                   'lecturer':[lecturer]})
        
            time.sleep(1)
            finals_df = pd.concat([finals_df,temp_df])
            click(S(".ui-icon-close"))
        
    else:
        
        for i in links:
            time.sleep(1)
            click(i)
            time.sleep(1)

            courseDay = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='اليوم'))
            courseTime = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='الوقت'))
            courseDay = [item.web_element.text for item in courseDay]
            courseTime = [item.web_element.text for item in courseTime]
            examDay = [item.web_element.text for item in find_all(S("#examDayDiv"))]
            examTime = [item.web_element.text for item in find_all(S("#examTimeDiv"))]
            examDate = [item.web_element.text for item in find_all(S("#examDateDiv"))]
            lecturer = [item.web_element.text for item in find_all(S("#instructorDiv"))]
            temp_df = pd.DataFrame({'course_day':[courseDay],'course_time':[courseTime],
            'final_exam_day':[examDay[0]], 'final_exam_date':[examDate[0]],'final_exam_time':[examTime[0]],
                                   'lecturer':[lecturer]})

            time.sleep(1)
            finals_df = pd.concat([finals_df,temp_df])
            click(S(".ui-icon-close"))
    """
    row = None
        
    counter +=1
    row1 = []
    row2  = []
    click(S(".pui-paginator-next"))


In [20]:
courses_df

,course_code,course_name,section_number,course_type,course_hours,course_gender
0,104 فيز,فيزياء عامه (2),81707,محاضرة,4,ذكر
0,104 فيز,فيزياء عامه (2),81708,عملي,ذكر,مغلقة
0,104 فيز,فيزياء عامه (2),81709,عملي,ذكر,مغلقة
0,104 فيز,فيزياء عامه (2),82652,محاضرة,4,ذكر
0,104 فيز,فيزياء عامه (2),82653,عملي,ذكر,مفتوحة
...,...,...,...,...,...,...
0,106 ريض,حساب التكامل,81659,محاضرة,3,ذكر
0,106 ريض,حساب التكامل,81660,تمارين,ذكر,مفتوحة
0,106 ريض,حساب التكامل,81661,محاضرة,3,ذكر
0,106 ريض,حساب التكامل,81662,تمارين,ذكر,مفتوحة
